### ML Validation Metrics. Part 2
#### Date: 2026-01-23

**Topics:**
> 1. Causality
> 2. Confounders
> 3. Calibration
> 4. LogLoss

**Materials:**
> 1. 
> 2.



#### Causality
ML-модель вміє передбачати, але НЕ вміє доводити вплив дії

**Prediction:** спрогнозувати proba для churn  
**Intervention:** запрононувати знижку 

##### prediction != intervention

ML-модель виявляє:
> транзакції з високою ймовірністю fraud

Помилковий висновок:
> Якщо ми їх заблокуємо — ми зменшимо fraud

Модель не знає, що було б:
> якби ми не заблокували \
> якби ми заблокували інакше

#### Confounders

Фактор, який впливає і на ознаку, і на результат
та створює хибне враження причинності


**Спостереження з даних:**
> користувачі зі знижками частіше йдуть

**Хибний висновок:**
> Знижки збільшують churn

**Реальність:**
> знижки дають тим, хто вже хотів піти

намір піти — confounder

#### Calibration 

**Модель видає:** p = 0.8, 80% імовірність -> можна діяти впевнено

> серед усіх прогнозів p~0.8 \
> реально fraud трапляється у 50% випадків \
> модель переоцінює ризик


Якщо модель каже p = 0.3,
то подія відбувається приблизно у 30% випадків

> бізнес приймає рішення по порогах \
> пороги базуються на p \
> якщо p бреше -> рішення хаотичні

##### LogLoss - індекатор калібровки
наскільки сильно модель карається за впевнені, але неправильні прогнози

Якщо дві моделі мають схожий PR-AUC, але одна має нижчий LogLoss — її ймовірностям можна більше довіряти

> ML -> оцінює ризик \
> Calibration -> робить ризик чесним \
> Policy -> перетворює ризик у дію \
> Causality -> не дозволяє брехати про ефект


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, log_loss,
    confusion_matrix
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
df = pd.read_csv("creditcard.csv")

missing_rate = df.isna().mean().sort_values(ascending=False)
print("\nTop missing columns:")
print(missing_rate.head(10))


class_counts = df["Class"].value_counts()
class_ratio = df["Class"].value_counts(normalize=True)

print("\nClass counts:\n", class_counts)
print("\nClass ratio:\n", class_ratio)


Top missing columns:
Time    0.0
V1      0.0
V2      0.0
V3      0.0
V4      0.0
V5      0.0
V6      0.0
V7      0.0
V8      0.0
V9      0.0
dtype: float64

Class counts:
 Class
'0'    284315
'1'       492
Name: count, dtype: int64

Class ratio:
 Class
'0'    0.998273
'1'    0.001727
Name: proportion, dtype: float64


In [4]:
df['Class'] = df['Class'].str.replace("'", "")

In [5]:
X = df.drop(columns=["Class"])
y = df["Class"].astype(int)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

In [7]:
def recall_at_k(y_true: pd.Series, y_proba: np.ndarray, k_frac: float) -> float:
    n = len(y_true)
    k = max(1, int(n * k_frac))

    top_idx = np.argsort(y_proba)[::-1][:k]
    fraud_in_top = y_true.iloc[top_idx].sum()

    total_fraud = y_true.sum()

    return fraud_in_top / max(1, total_fraud)

In [8]:
def evaluate_all(y_true: pd.Series, y_proba: np.ndarray, threshold: float, k_fracs=(0.001, 0.005, 0.01)) -> dict:
    y_pred = (y_proba >= threshold).astype(int)

    out = {
        "threshold": threshold,
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "ROC-AUC": roc_auc_score(y_true, y_proba),
        "PR-AUC": average_precision_score(y_true, y_proba),
        "LogLoss": log_loss(y_true, y_proba, labels=[0, 1]),
        "ConfusionMatrix": confusion_matrix(y_true, y_pred),
    }

    for kf in k_fracs:
        out[f"Recall@{kf*100:.2f}%"] = recall_at_k(y_true, y_proba, k_frac=kf)

    return out

In [9]:
def print_eval(title: str, metrics: dict):
    print("\n" + "="*60)
    print(title)
    print("="*60)
    for k, v in metrics.items():
        if k == "ConfusionMatrix":
            continue
        print(f"{k:>14}: {v}")

    cm = metrics["ConfusionMatrix"]
    tn, fp, fn, tp = cm.ravel()
    print("\nConfusion Matrix [[TN FP],[FN TP]]:")
    print(cm)
    print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")

In [10]:
lr = Pipeline(steps=[
    ("scaler", StandardScaler()),  
    ("model", LogisticRegression(
        class_weight = "balanced",   
        max_iter = 2000
    ))
])

lr.fit(X_train, y_train)

proba_lr = lr.predict_proba(X_test)[:, 1]
res_lr_05 = evaluate_all(y_test, proba_lr, threshold=0.5)
print_eval("LogReg threshold = 0.5", res_lr_05)


LogReg threshold = 0.5
     threshold: 0.5
     Precision: 0.06097560975609756
        Recall: 0.9183673469387755
            F1: 0.11435832274459974
       ROC-AUC: 0.9720834996210077
        PR-AUC: 0.7189705771419241
       LogLoss: 0.11219568508756639
  Recall@0.10%: 0.45918367346938777
  Recall@0.50%: 0.8877551020408163
  Recall@1.00%: 0.8979591836734694

Confusion Matrix [[TN FP],[FN TP]]:
[[55478  1386]
 [    8    90]]
TN=55478, FP=1386, FN=8, TP=90


In [11]:
rf = RandomForestClassifier(
    n_estimators = 400,
    random_state = RANDOM_STATE,
    n_jobs = -1,
    class_weight = "balanced_subsample"  
)
rf.fit(X_train, y_train)
proba_rf = rf.predict_proba(X_test)[:, 1]

res_rf_05 = evaluate_all(y_test, proba_rf, threshold=0.5)
print_eval("RandomForest threshold = 0.5", res_rf_05)


RandomForest threshold = 0.5
     threshold: 0.5
     Precision: 0.961038961038961
        Recall: 0.7551020408163265
            F1: 0.8457142857142858
       ROC-AUC: 0.9561622324084388
        PR-AUC: 0.8635382284721764
       LogLoss: 0.00631719530271903
  Recall@0.10%: 0.5510204081632653
  Recall@0.50%: 0.8979591836734694
  Recall@1.00%: 0.8979591836734694

Confusion Matrix [[TN FP],[FN TP]]:
[[56861     3]
 [   24    74]]
TN=56861, FP=3, FN=24, TP=74


In [12]:
param_dist = {
    "n_estimators": [300, 500, 800],
    "max_depth": [None, 8, 12, 16, 20],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 5, 10],
    "max_features": ["sqrt", "log2", None]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

search = RandomizedSearchCV(
    estimator=RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_jobs=-1,
        class_weight="balanced_subsample"
    ),
    param_distributions=param_dist,
    n_iter=50,                      
    scoring="average_precision",     
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
rf_tuned = search.best_estimator_
print("\nBest params:", search.best_params_)
print("Best CV PR-AUC:", search.best_score_)

proba_rf_tuned = rf_tuned.predict_proba(X_test)[:, 1]
res_rf_tuned_05 = evaluate_all(y_test, proba_rf_tuned, threshold=0.5)
print_eval("RandomForest (tuned) @ threshold=0.5", res_rf_tuned_05)

print("\nRecall@0.5% (RF tuned):", recall_at_k(y_test, proba_rf_tuned, 0.005))



In [ ]:
rf_cal = CalibratedClassifierCV(
    estimator=rf_tuned,   
    method="isotonic",
    cv=3
)

rf_cal.fit(X_train, y_train)
proba_rf_cal = rf_cal.predict_proba(X_test)[:, 1]


res_before = evaluate_all(y_test, proba_rf_tuned, threshold=0.2)
res_after  = evaluate_all(y_test, proba_rf_cal, threshold=0.2)

print_eval("RF tuned BEFORE calibration  threshold=0.2", res_before)
print_eval("RF tuned AFTER  calibration  threshold=0.2", res_after)


In [21]:
def plot_calibration(y_true, prob_uncal, prob_cal, n_bins=10):
    t_uncal, p_uncal = calibration_curve(y_true, prob_uncal, n_bins=n_bins, strategy="uniform")
    t_cal,   p_cal   = calibration_curve(y_true, prob_cal,   n_bins=n_bins, strategy="uniform")

    plt.figure()
    plt.plot([0, 1], [0, 1])

    plt.plot(p_uncal, t_uncal, marker="o", label="Before calibration")

    plt.plot(p_cal, t_cal, marker="o", label="After calibration")

    plt.xlabel("Середня передбачена ймовірність (p)")
    plt.ylabel("Реальна частота fraud")
    plt.title("Calibration curve (reliability)")
    plt.legend()
    plt.show()

plot_calibration(y_test, proba_rf_tuned, proba_rf_cal, n_bins=10)


NameError: name 'proba_rf_tuned' is not defined

In [ ]:
def fraud_policy(p: float) -> str:
    if p >= 0.90:
        return "AUTO_BLOCK_or_2FA"
    elif p >= 0.40:
        return "MANUAL_REVIEW"
    else:
        return "ALLOW"

actions = pd.Series(proba_rf_cal).apply(fraud_policy)

print("\nAction distribution:")
print(actions.value_counts(normalize=True))